In [2]:
from numpy import *
def loadSimpData():
    datMat = matrix([
        [1,2.1],
        [2,1.1],
        [1.3,1],
        [1,1],
        [2,1]
    ])
    classLabels = [1,1,-1,-1,1]
    return datMat,classLabels


In [17]:
#单层决策树生成函数
def stumpClassify(dataMatrix,dimen,threshVal,threshIneq):
    retArray = ones((shape(dataMatrix)[0],1))
    if threshIneq == 'lt':
        retArray[dataMatrix[:,dimen]<=threshVal] = -1.0
    else:
        retArray[dataMatrix[:,dimen]>threshVal] =-1.0
    return retArray


def buildStump(dataArr,classLabels,D):
    dataMatrix = mat(dataArr); labelMat = mat(classLabels).T
    m,n = shape(dataMatrix)
    numSteps = 10; bestStump = {}; bestClasEst = mat(zeros((m,1)))
    minError = inf
    for i in range(n):
        rangeMin = dataMatrix[:,i].min(); rangeMax = dataMatrix[:,i].max();
        stepSize = (rangeMax-rangeMin)/numSteps
        for j in range(-1,int(numSteps)+1):
            for inequal in ['lt','gt']:
                threshVal = (rangeMin+float(j)*stepSize)
                predictedVals = stumpClassify(dataMatrix,i,threshVal,inequal)
                errArr = mat(ones((m,1)))
                errArr[predictedVals==labelMat]=0
                weightedError = D.T*errArr #alpha是分类器的权重alpha=0.5log((1-errorrate)/errorrate),D是（m,1）向量
                if weightedError<minError:  #D是样本的权重，D(t+1)=D(t)exp(alpha)/sum(D)
                    minError = weightedError
                    bestClasEst = predictedVals.copy()
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq']= inequal
    return bestStump,minError,bestClasEst

In [20]:
#基于单层决策树的Adaboost训练过程
def adaBoostTrainDS(dataArr,classLabels,numIt=40):
    weakClassArr = []
    m = shape(dataArr)[0]
    D = mat(ones((m,1))/m)
    aggClassEst = mat(zeros((m,1)))
    for i in range(numIt):
        bestStump,error,classEst = buildStump(dataArr,classLabels,D)
        print ("D:",D.T)
        alpha = float(0.5*log((1.0-error)/max(error-1e-16)))#alpha是分类器的权重alpha=0.5log((1-errorrate)/errorrate)
        bestStump['alpha'] = alpha#alpha是个数值，error是个数值
        weakClassArr.append(bestStump)
        print ("classEst:",classEst.T)
        expon = multiply(-1*alpha*mat(classLabels).T,classEst) #expon是个(m,1)向量
        D = multiply(D,exp(expon)) #multiply:elementbyelement
        D = D/D.sum()       #D是样本的权重，D(t+1)=D(t)exp(alpha)/sum(D),D是（m,1）向量
        aggClassEst +=alpha*classEst
        print ("aggClassEst:",aggClassEst.T)
        aggErrors = multiply(sign(aggClassEst)!=mat(classLabels).T,ones((m,1)))#classEst(m,1),aggClassEst(m,1)
        errorRate = aggErrors.sum()/m
        print ('total error:',errorRate,'\n')
        if errorRate ==0.0: break
    return weakClassArr
    

In [21]:
datMat,classLabels = loadSimpData()
adaBoostTrainDS(datMat,classLabels)

D: [[0.2 0.2 0.2 0.2 0.2]]
classEst: [[-1.  1. -1. -1.  1.]]
aggClassEst: [[-0.69314718  0.69314718 -0.69314718 -0.69314718  0.69314718]]
total error: 0.2 

D: [[0.5   0.125 0.125 0.125 0.125]]
classEst: [[ 1.  1. -1. -1. -1.]]
aggClassEst: [[ 0.27980789  1.66610226 -1.66610226 -1.66610226 -0.27980789]]
total error: 0.2 

D: [[0.28571429 0.07142857 0.07142857 0.07142857 0.5       ]]
classEst: [[1. 1. 1. 1. 1.]]
aggClassEst: [[ 1.17568763  2.56198199 -0.77022252 -0.77022252  0.61607184]]
total error: 0.0 



[{'dim': 0, 'thresh': 1.3, 'ineq': 'lt', 'alpha': 0.6931471805599456},
 {'dim': 1, 'thresh': 1.0, 'ineq': 'lt', 'alpha': 0.9729550745276572},
 {'dim': 0, 'thresh': 0.9, 'ineq': 'lt', 'alpha': 0.8958797346140284}]

In [22]:
#adaboost分类函数
def adaClassify(datToClass,classifierArr):
    dataMatrix = mat(datToClass)
    m = shape(dataMatrix)[0]
    aggClassEst = mat(zeros((m,1)))
    for i in range(len(classifierArr)):
        classEst = stumpClassify(dataMatrix,classifierArr[i]['dim'],classifierArr[i]['thresh'],classifierArr[i]['ineq'])
        aggClassEst += classifierArr[i]['alpha']*classEst#随着迭代的进行，数据点的分类结果越来越强
        print (aggClassEst)
    return sign(aggClassEst)

In [23]:
classifierArr = adaBoostTrainDS(datMat,classLabels,30)
adaClassify([0,0],classifierArr)

D: [[0.2 0.2 0.2 0.2 0.2]]
classEst: [[-1.  1. -1. -1.  1.]]
aggClassEst: [[-0.69314718  0.69314718 -0.69314718 -0.69314718  0.69314718]]
total error: 0.2 

D: [[0.5   0.125 0.125 0.125 0.125]]
classEst: [[ 1.  1. -1. -1. -1.]]
aggClassEst: [[ 0.27980789  1.66610226 -1.66610226 -1.66610226 -0.27980789]]
total error: 0.2 

D: [[0.28571429 0.07142857 0.07142857 0.07142857 0.5       ]]
classEst: [[1. 1. 1. 1. 1.]]
aggClassEst: [[ 1.17568763  2.56198199 -0.77022252 -0.77022252  0.61607184]]
total error: 0.0 

[[-0.69314718]]
[[-1.66610226]]
[[-2.56198199]]


matrix([[-1.]])